<a href="https://colab.research.google.com/github/DmitryAlexandrovv/infopoisk/blob/master/task4/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=28eb89ba8497399af929b866760a84eaa70f8832022695c05e13a4b84c5287f1
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [3]:
import math
import string
import pymorphy2
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')

def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

def get_text_from_html(html):
    soup = BeautifulSoup(data, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text(' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    # Убираем предлоги и союзы, знаки препинания
    words = text.split()
    functors_pos = {'CONJ', 'PREP'}  # function words
    tt = str.maketrans(dict.fromkeys(string.punctuation))
    prepared_text = ' '.join([word for word in words if pos(word) not in functors_pos]).translate(tt)

    return prepared_text

tokens_file = open('drive/MyDrive/dz/tokens.txt', 'r')
lemmas_file = open('drive/MyDrive/dz/lemmas.txt', 'r')
lemmas_lines = lemmas_file.readlines()
lines = tokens_file.readlines()
dictionary_tokens = {}
dictionary_lemmas = {}

files_content = {}

for i in range(100):
    with open('drive/MyDrive/dz/pages/page' + str(i) + '.html', 'r') as file:
        data = file.read()
        text = get_text_from_html(data)

        files_content.update({str(i): text})

for line in lemmas_lines:
    lemma = line.strip().partition(': ')[0]
    forms = line.strip().partition(': ')[2]
    pages_count_with_lemma = 0
    
    for file in files_content:
        text = files_content.get(file)
        words_count = len(text.split(' '))
        forms_count = 0

        for form in forms:
            if text.lower().find(form) != -1 and len(form.strip()) > 0:
                forms_count += 1
                break

        for form in forms.partition(' '):
            if text.lower().find(form) != -1 and len(form.strip()) > 0:
                pages_count_with_lemma += 1
                break

        
        if (dictionary_lemmas.get(file)):
            dictionary_lemmas.get(file).update({lemma: {"forms_count": forms_count, "words_count": words_count, 'pages_count_with_lemma': pages_count_with_lemma}})
        else:
            dictionary_lemmas.update({file: {lemma: {"forms_count": forms_count, "words_count": words_count, 'pages_count_with_lemma': pages_count_with_lemma}}})

for line in lines:
    token = line.strip()
    tokens = {}
    pages_count_with_token = 0

    for file in files_content:
        text = files_content.get(file)
        if (text.lower().find(token) != -1):
            pages_count_with_token += 1

    for file in files_content:
        text = files_content.get(file)
        count = text.lower().count(token)
        lemmas_count = 0
        words_count = len(text.split(' '))
            
        tokens.update({token: {'tf': count, 'words_count': words_count, "pages_count_with_token": pages_count_with_token}})

        if (dictionary_tokens.get(file)):
            dictionary_tokens.get(file).get('tokens').update(tokens)
        else:
            dictionary_tokens.update({file: {'tokens': tokens}})

tokens_output = 'drive/MyDrive/dz/tokens_tf_idf/'
lemmas_output = 'drive/MyDrive/dz/lemmas_tf_idf/'

for value in dictionary_tokens:
    page_dictionary = dictionary_tokens.get(value)
    tokens = page_dictionary.get('tokens')
    
    with open(tokens_output + 'page-' + value + '.txt', 'w') as writefile:
        for token in tokens:
            idf = math.log2(100 / tokens.get(token).get('pages_count_with_token'))
            tf = tokens.get(token).get('tf') / tokens.get(token).get('words_count')
            writefile.write(token + " " + str(idf) + " " + str(tf * idf) + "\n")

for value in dictionary_lemmas:
    lemmas = dictionary_lemmas.get(value)
    with open(lemmas_output + 'page-' + value + '.txt', 'w') as writefile:
        for lemma in lemmas:
            if (lemmas.get(lemma).get('pages_count_with_lemma') == 0):
                idf = 0.0
            else:
                idf = math.log2(100 / lemmas.get(lemma).get('pages_count_with_lemma'))

            tf = lemmas.get(lemma).get('forms_count') / lemmas.get(lemma).get('words_count')
            writefile.write(lemma + " " + str(idf) + " " + str(tf * idf) + "\n")


Mounted at /content/drive
